In [1]:
# This sript is used for transforming words into vectors, and plot them on x-y scatter plots

# Before using, words should be processed by following steps
## Sentences of given articles are first breaked into words by Jieba
## Words are pooled in a list (a bag of word)
## Stop words are removed from word pools during processing
## Processed words are concatenated to from a long string, stroed in a sqlite3 database


import sqlite3
from gensim.models import word2vec
from gensim.models import KeyedVectors

# Remove words appearing only once (optional)
def corpus_process(a,b):
    print("Run corpus_process")
    input_list = a
    frequency_threshold = b
    from collections import defaultdict
    frequency = defaultdict(int)
    for token in input_list:
        frequency[token] += 1
            
    return_list=[]
    for token in input_list:
        if frequency[token] > frequency_threshold:
            return_list.append(token)
    
    return(return_list)
    
def main():
    print("Run main")
    
    # Read article content from database
    conn = sqlite3.connect('ptt.db')
    c = conn.cursor()
    
    cursor = c.execute("SELECT id, content from test")
    articleid = []
    articlecontent = []
    for row in cursor:
        articleid.append(row[0])
        articlecontent.append(row[1].split(','))
    
    frequency_threshold = input("Enter the frequency threshold of words for training:")
    processed_corpus=[]
    i = 0
    while i < len(articlecontent):
        processed_corpus.append(corpus_process(articlecontent[i], int(frequency_threshold)))
        i+=1
    
    merged_processed_corpus = []
    j = 0
    # Combine all words in the corpus
    while j < len(processed_corpus):
        merged_processed_corpus+=processed_corpus[j]
        j+=1
    print(len(merged_processed_corpus))
    
    # Write all words in txt file
    k = 0
    txtfile = "processed_corpus.txt"
    with open(txtfile,'w') as writer:
        while k < len(merged_processed_corpus):
            writer.write(merged_processed_corpus[k])
            writer.write('\n')
            k+=1
    print("write file done")
    
    sentences = word2vec.LineSentence('processed_corpus.txt')
    for number in [50,100,200,300]:
        model = word2vec.Word2Vec(sentences,size=number,window=5,min_count=1,workers=20)
        # min-count is the frequency of a given word in the corpus, default setting is 5. If the frequency of this words is < 5, then it will be abadoned. 
        # size = vector space
        # The vector format should be consistent with glove vector
        model.wv.save_word2vec_format('word2vec'+str(number)+'.txt', binary=False)
        
    for number in [50,100,200,300]:
        wv = KeyedVectors.load_word2vec_format('word2vec'+ str(number)+'.txt', binary=False)
        print(number, wv.similarity('北九州','前往')) #两个词的相关性
    
    
    
    
    
    
if __name__ == "__main__":
    print("initialize")
    main() 

initialize
Run main
Enter the frequency threshold of words for training:1
Run corpus_process
Run corpus_process
398
write file done
50 -0.1606269
100 -0.07849868
200 -0.08044776
300 -0.13878857


# 